In [164]:
import pandas as pd
import re
import string
import jieba
import collections

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [165]:
# all_question = pd.read_csv('./data/description.csv')
all_question = pd.read_excel('./data/description.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: './data/description.xlsx'

In [ ]:
all_question.head(20)

In [ ]:
all_question.dropna(axis=0, inplace=True, subset=['Description'])

In [ ]:
all_question.head(10)

In [ ]:
all_question.isna().sum()

In [ ]:
all_question[pd.isnull(all_question).any(axis=1)]

In [ ]:
# 去除所有“[]”内的文本
def remove_special_string(text):
#     return text.replace("[Wechat]", "")
    text = re.sub(r'\[[\s\S]*\]', '', text)
    text = re.sub(r'\([\s\S]*\)', '', text)
    text = re.sub(r'\<[\s\S]*\>', '', text)
    return text
def remove_stopwords(text):
    text = text.replace("赫赛汀", "")
    text = text.replace("患者", "")
    text = text.replace("咨询", "")
    text = text.replace("告知", "")
    text = text.replace("家属", "")
    text = text.replace("医生", "")

In [166]:
all_question['Description'] = all_question.Description.apply(lambda x: remove_special_string(x))

In [167]:
# all_question.to_excel('./data/after_re_sub.xlsx')
all_question.head()

,Case Number,Description
0,1420656.0,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配..."
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）
2,1434184.0,请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）
3,1436011.0,拿到的赫赛汀呈现块状是否正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用


In [168]:
jieba.load_userdict('./jieba_dict.txt')

In [169]:
demo_wordcut = jieba.cut('赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。', cut_all=False)
print(' '.join(demo_wordcut).split())

['赫赛汀', '（', '批号', '1', '：', 'N3825B03B3049', ',', '批号', '2', '：', 'N3826B02B3050', '）', '配', '了', '之后', '结冰', '了', '，', '能否', '继续', '使用', '。']


In [170]:
len(all_question)

757

In [171]:
all_text_data = []
for idx,row in all_question.iterrows():
    all_text_data.append(row['Description'])

In [172]:
len(all_text_data)

757

In [173]:
chinese_punc = string.punctuation + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."""
text = "赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。"
text.translate(str.maketrans('', '', chinese_punc))

'赫赛汀 批号1\xa0N3825B03B3049 批号2 N3826B02B3050 配了之后结冰了 能否继续使用'

In [174]:
chinese_punc = string.punctuation + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."""
def jieba_cut(text):
    text = text.translate(str.maketrans('', '', chinese_punc))
    return ' '.join(jieba.cut(text, cut_all=False)).split()

In [175]:
# term-tfidf matrix
tfidf_vectorizer = TfidfVectorizer(input='content', tokenizer=jieba_cut, use_idf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(all_text_data)

In [176]:
tfidf_matrix

<757x2119 sparse matrix of type '<class 'numpy.float64'>'
	with 10490 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_vectorizer.vocabulary_

In [281]:
kmeans_model = KMeans(n_clusters=40, max_iter=400)
kmeans_model.fit(tfidf_matrix)
kmeans_clustering = collections.defaultdict(list)
for idx, label in enumerate(kmeans_model.labels_):
    kmeans_clustering[label].append(idx)

In [282]:
f = open("./kmeans_demo.txt", 'w')
for key, value in kmeans_clustering.items():
    for idx in value:
#         print(all_text_data[idx])
        f.write(all_text_data[idx] + '\n')
    f.write('\n-------------------------------------------------------------------------------------------------------------\n\n')
f.close()

# for idx in kmeans_clustering[30]:
#     print(all_text_data[idx])

In [339]:
dbscan_model = DBSCAN(eps=0.01, min_samples=10)
dbscan_model.fit(tfidf_matrix)
dbscan_clustering = collections.defaultdict(list)
for idx, label in enumerate(dbscan_model.labels_):
    dbscan_clustering[label].append(idx)

In [342]:
print(len(set(dbscan_clustering)))
for idx in dbscan_clustering[2]:
    print(all_text_data[idx])

3
